In [2]:
import smtplib
from email.mime.text import MIMEText


In [10]:
import requests
import time
import json
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt
import smtplib
from email.mime.text import MIMEText

# OpenWeatherMap API Configuration
API_KEY = '2bff56c00d1e2e022b04a3f354e94c26'  # Replace with your actual API key from OpenWeatherMap
BASE_URL = 'http://api.openweathermap.org/data/2.5/weather'
FORECAST_URL = 'http://api.openweathermap.org/data/2.5/forecast'  # For forecasts
CITIES = ['Delhi', 'Mumbai', 'Chennai', 'Bangalore', 'Kolkata', 'Hyderabad']

# Define units (metric = Celsius)
UNITS = 'metric'  # or 'imperial' for Fahrenheit
ALERT_THRESHOLD_TEMP = 35.0  # Alert if temp exceeds 35°C
CONSECUTIVE_ALERT_COUNT = 2  # Trigger alert after two consecutive updates breaching the threshold


In [11]:
# Function to get current weather data for a city
def get_weather(city):
    url = f"{BASE_URL}?q={city}&appid={API_KEY}&units={UNITS}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        return {
            'city': city,
            'main': data['weather'][0]['main'],
            'temp': data['main']['temp'],
            'feels_like': data['main']['feels_like'],
            'humidity': data['main']['humidity'],  # Include additional parameter
            'wind_speed': data['wind']['speed'],   # Include additional parameter
            'dt': datetime.now()  # Use system time to capture real-time updates
        }
    else:
        print(f"Error retrieving data for {city}: {response.status_code}")
        return None

# Function to fetch weather data at a set interval (e.g., every 5 minutes)
def fetch_weather_data(interval=300):
    weather_data = []
    while True:
        for city in CITIES:
            try:
                data = get_weather(city)
                if data:
                    print(f"Weather data fetched for {city}: {data}")
                    weather_data.append(data)
            except Exception as e:
                print(f"Error fetching data for {city}: {e}")
        
        # Process data and check alerts after every interval
        if len(weather_data) > 0:
            process_weather_data(weather_data)
        
        time.sleep(interval)


In [12]:
# Function to calculate daily weather summary (rollups and aggregates)
def calculate_daily_summary(weather_data):
    df = pd.DataFrame(weather_data)
    df['date'] = df['dt'].dt.date  # Extract date

    # Group by city and date for daily rollup
    daily_summary = df.groupby(['city', 'date']).agg({
        'temp': ['mean', 'max', 'min'],
        'humidity': 'mean',
        'wind_speed': 'mean',
        'main': lambda x: x.mode()[0]  # Dominant weather condition (most frequent)
    }).reset_index()

    # Rename columns for clarity
    daily_summary.columns = ['city', 'date', 'avg_temp', 'max_temp', 'min_temp', 'avg_humidity', 'avg_wind_speed', 'dominant_weather']
    
    # Save the daily summary for future reference
    daily_summary.to_csv('daily_weather_summary.csv', index=False)
    
    return daily_summary

# Function to process and roll up weather data at the end of the day
def process_weather_data(weather_data):
    daily_summary = calculate_daily_summary(weather_data)
    print(daily_summary)
    weather_data.clear()  # Clear the data for the next day's collection


In [13]:
# Function to send email alerts when thresholds are breached
def send_alert(city, temp, threshold_temp):
    msg = MIMEText(f"Alert! {city} temperature exceeded {threshold_temp}°C. Current temperature: {temp}°C")
    msg['Subject'] = f"Weather Alert for {city}"
    msg['From'] = "your_email@example.com"
    msg['To'] = "recipient_email@example.com"

    with smtplib.SMTP('smtp.example.com', 587) as server:
        server.starttls()
        server.login("your_email@example.com", "your_password")
        server.send_message(msg)
    print(f"Email alert sent for {city}!")

# Function to check for temperature alerts
def check_alerts(weather_data, threshold_temp=ALERT_THRESHOLD_TEMP, consecutive_count=CONSECUTIVE_ALERT_COUNT):
    city_alerts = {}
    
    df = pd.DataFrame(weather_data).sort_values(by=['city', 'dt'])
    
    for city in CITIES:
        city_data = df[df['city'] == city]
        consecutive_breaches = 0
        
        for _, row in city_data.iterrows():
            if row['temp'] > threshold_temp:
                consecutive_breaches += 1
                if consecutive_breaches >= consecutive_count:
                    print(f"Alert! {city} has exceeded {threshold_temp}°C for {consecutive_count} consecutive updates.")
                    send_alert(city, row['temp'], threshold_temp)
            else:
                consecutive_breaches = 0  # Reset if threshold not breached


In [14]:
def plot_temperature_trends(summary_data):
    for city in CITIES:
        city_data = summary_data[summary_data['city'] == city]
        plt.plot(city_data['date'], city_data['avg_temp'], label=f'{city} Avg Temp')
        plt.fill_between(city_data['date'], city_data['min_temp'], city_data['max_temp'], alpha=0.2)
    
    plt.xlabel('Date')
    plt.ylabel('Temperature (°C)')
    plt.title('Daily Temperature Trends')
    plt.legend()
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()


In [15]:
def plot_alerts(alert_data):
    alert_df = pd.DataFrame(alert_data)
    alert_df['time'] = alert_df['dt'].dt.time  # Extract time for alerts

    plt.scatter(alert_df['time'], alert_df['temp'], color='red', label='Temperature Alerts')
    plt.xlabel('Time')
    plt.ylabel('Temperature')
    plt.title('Temperature Alerts Over Time')
    plt.show()


In [16]:
# Function to get weather forecast data for a city
def get_forecast(city):
    url = f"{FORECAST_URL}?q={city}&appid={API_KEY}&units={UNITS}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        forecast_data = []
        for entry in data['list']:
            forecast_data.append({
                'city': city,
                'temp': entry['main']['temp'],
                'humidity': entry['main']['humidity'],
                'wind_speed': entry['wind']['speed'],
                'main': entry['weather'][0]['main'],
                'dt': datetime.fromtimestamp(entry['dt'])
            })
        return forecast_data
    else:
        print(f"Error retrieving forecast for {city}: {response.status_code}")
        return None

# Function to fetch forecasts for all cities
def fetch_forecast_data():
    forecast_data = []
    for city in CITIES:
        data = get_forecast(city)
        if data:
            forecast_data.extend(data)
    return forecast_data

# Example of fetching forecast data
forecast_data = fetch_forecast_data()
print(forecast_data)


[{'city': 'Delhi', 'temp': 34.59, 'humidity': 24, 'wind_speed': 2.54, 'main': 'Clear', 'dt': datetime.datetime(2024, 10, 18, 17, 30)}, {'city': 'Delhi', 'temp': 32.01, 'humidity': 30, 'wind_speed': 2.68, 'main': 'Clear', 'dt': datetime.datetime(2024, 10, 18, 20, 30)}, {'city': 'Delhi', 'temp': 29.12, 'humidity': 36, 'wind_speed': 2.67, 'main': 'Clear', 'dt': datetime.datetime(2024, 10, 18, 23, 30)}, {'city': 'Delhi', 'temp': 27.67, 'humidity': 38, 'wind_speed': 1.97, 'main': 'Clear', 'dt': datetime.datetime(2024, 10, 19, 2, 30)}, {'city': 'Delhi', 'temp': 26.1, 'humidity': 43, 'wind_speed': 2.59, 'main': 'Clear', 'dt': datetime.datetime(2024, 10, 19, 5, 30)}, {'city': 'Delhi', 'temp': 27.51, 'humidity': 40, 'wind_speed': 2.84, 'main': 'Clear', 'dt': datetime.datetime(2024, 10, 19, 8, 30)}, {'city': 'Delhi', 'temp': 32.74, 'humidity': 26, 'wind_speed': 1.49, 'main': 'Clear', 'dt': datetime.datetime(2024, 10, 19, 11, 30)}, {'city': 'Delhi', 'temp': 35.72, 'humidity': 19, 'wind_speed': 0.

In [ ]:
# Start real-time data monitoring
fetch_weather_data(interval=300)  # Fetch data every 5 minutes


Weather data fetched for Delhi: {'city': 'Delhi', 'main': 'Haze', 'temp': 33.05, 'feels_like': 32.53, 'humidity': 33, 'wind_speed': 2.57, 'dt': datetime.datetime(2024, 10, 18, 18, 2, 37, 960428)}
Weather data fetched for Mumbai: {'city': 'Mumbai', 'main': 'Haze', 'temp': 29.99, 'feels_like': 36.01, 'humidity': 74, 'wind_speed': 5.66, 'dt': datetime.datetime(2024, 10, 18, 18, 2, 38, 144782)}
Weather data fetched for Chennai: {'city': 'Chennai', 'main': 'Mist', 'temp': 28.8, 'feels_like': 35.47, 'humidity': 86, 'wind_speed': 1.03, 'dt': datetime.datetime(2024, 10, 18, 18, 2, 38, 445201)}
Weather data fetched for Bangalore: {'city': 'Bangalore', 'main': 'Clouds', 'temp': 26.79, 'feels_like': 28.58, 'humidity': 71, 'wind_speed': 1.64, 'dt': datetime.datetime(2024, 10, 18, 18, 2, 38, 766299)}
Weather data fetched for Kolkata: {'city': 'Kolkata', 'main': 'Rain', 'temp': 28.04, 'feels_like': 31.89, 'humidity': 78, 'wind_speed': 1.89, 'dt': datetime.datetime(2024, 10, 18, 18, 2, 38, 977632)}
W